<a href="https://colab.research.google.com/github/thomson008/pdiot-cw3/blob/main/Data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from scipy import stats
import keras
from keras.layers import Conv1D, Dropout, Dense, MaxPooling1D, Flatten
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/My Drive/Colab Notebooks/pdiot-cw3/data'

file_list = []
sensor_position = 'Chest_Right'

for root, dirs, files in os.walk(path):
    for file in files:
        if ('csv' in file and sensor_position in file):
            file_list.append(os.path.join(root, file))
            
dataframes = []

for file in file_list:
    with open(file) as f:
        info = [next(f).rstrip().split(': ')[1] for x in range(5)]
        user_id = info[4]
        activity = info[2]
        
    file_data = pd.read_csv(file, header=5)
    if (file_data.size):
        dataframes.append(file_data)
        file_data.insert(0, 'user_id', user_id)
        file_data.insert(1, 'activity', activity)
    

sensor_data = pd.concat(dataframes, ignore_index=True).drop('seq', axis=1)
sensor_data.to_csv('sensor.csv')

In [19]:
def split_data():
    subjects = sensor_data.user_id.unique()
    train_subjects, test_subjects = subjects[: int(0.98*len(subjects))], subjects[int(0.98*len(subjects)):]

    df_train, df_test = sensor_data[sensor_data.user_id != 's1758009'], sensor_data[sensor_data.user_id == 's1758009']

    scale_columns = ['accel_x', 'accel_y', 'accel_z']

    scaler = RobustScaler()

    scaler = scaler.fit(df_train[scale_columns])

    df_train.loc[:, scale_columns] = scaler.transform(
      df_train.loc[:, scale_columns].to_numpy()
    )

    df_test.loc[:, scale_columns] = scaler.transform(
      df_test.loc[:, scale_columns].to_numpy()
    )

    return df_train, df_test

In [ ]:
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

TIME_STEPS = 36
STEP = 10

def create_whole_dataset(time_steps, step):
    df_train, df_test = split_data()
    X_train, y_train = create_dataset(
        df_train[scale_columns],
        df_train.activity,
        time_steps,
        step
    )

    X_test, y_test = create_dataset(
        df_test[scale_columns],
        df_test.activity,
        time_steps,
        step
    )

    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    enc = enc.fit(y_train)

    y_train = enc.transform(y_train)
    y_test = enc.transform(y_test)

create_whole_dataset(TIME_STEPS, STEP)

In [24]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = keras.Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

evaluate_model(X_train, y_train, X_test, y_test)

Epoch 1/10
212/212 [==============================] - 1s 7ms/step - loss: 1.2788 - accuracy: 0.5553
Epoch 2/10
212/212 [==============================] - 1s 7ms/step - loss: 0.7685 - accuracy: 0.7150
Epoch 3/10
212/212 [==============================] - 1s 7ms/step - loss: 0.6170 - accuracy: 0.7760
Epoch 4/10
212/212 [==============================] - 1s 7ms/step - loss: 0.5254 - accuracy: 0.8084
Epoch 5/10
212/212 [==============================] - 1s 7ms/step - loss: 0.4663 - accuracy: 0.8322
Epoch 6/10
212/212 [==============================] - 1s 7ms/step - loss: 0.4053 - accuracy: 0.8541
Epoch 7/10
212/212 [==============================] - 1s 7ms/step - loss: 0.3731 - accuracy: 0.8610
Epoch 8/10
212/212 [==============================] - 1s 6ms/step - loss: 0.3441 - accuracy: 0.8796
Epoch 9/10
212/212 [==============================] - 1s 7ms/step - loss: 0.3239 - accuracy: 0.8830
Epoch 10/10
212/212 [==============================] - 1s 7ms/step - loss: 0.3096 - accuracy: 0.8858

0.7363465428352356